<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ резюме из HeadHunter
   

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

: 

# Исследование структуры данных

1. Прочитайте данные с помощью библиотеки Pandas. Совет: перед чтением обратите внимание на разделитель внутри файла. 

In [ ]:
df = pd.read_csv('dst-3.0_16_1_hh_database.csv', sep=';')
print(f'Размерность таблицы: {df.shape}')

2. Выведите несколько первых (последних) строк таблицы, чтобы убедиться в том, что ваши данные не повреждены. Ознакомьтесь с признаками и их структурой.

In [ ]:
print("Первые 5 строк:")
print(df.head())
print("\nПоследние 5 строк:")
print(df.tail())
print("\nСтруктура признака 'Пол, возраст':")
print(df['Пол, возраст'].head(10))

In [ ]:
df.info()


3. Выведите основную информацию о числе непустых значений в столбцах и их типах в таблице.

4. Обратите внимание на информацию о числе непустых значений.

In [ ]:
print("Признаки с пропусками:")
print(df.isnull().sum())
print("\nПризнаки, где есть пропуски:")
columns_with_nulls = df.columns[df.isnull().any()].tolist()
print(columns_with_nulls)

5. Выведите основную статистическую информацию о столбцах.


In [ ]:
print("Основная статистическая информация:")
print(df.describe(include='all'))
print("\nУникальные значения в столбце 'Опыт работы':")
print(f"Количество уникальных значений: {df['Опыт работы'].nunique()}")
print("\nСамые распространенные должности:")
print(df['Ищет работу на должность:'].value_counts().head(10))

# Преобразование данных

1. Начнем с простого - с признака **"Образование и ВУЗ"**. Его текущий формат это: **<Уровень образования год выпуска ВУЗ специальность...>**. Например:
* Высшее образование 2016 Московский авиационный институт (национальный исследовательский университет)...
* Неоконченное высшее образование 2000  Балтийская государственная академия рыбопромыслового флота…
Нас будет интересовать только уровень образования.

Создайте с помощью функции-преобразования новый признак **"Образование"**, который должен иметь 4 категории: "высшее", "неоконченное высшее", "среднее специальное" и "среднее".

Выполните преобразование, ответьте на контрольные вопросы и удалите признак "Образование и ВУЗ".

Совет: обратите внимание на структуру текста в столбце **"Образование и ВУЗ"**. Гарантируется, что текущий уровень образования соискателя всегда находится в первых 2ух слов и начинается с заглавной буквы. Воспользуйтесь этим.

*Совет: проверяйте полученные категории, например, с помощью метода unique()*


In [ ]:
def get_education_level(text):
    if pd.isna(text):
        return np.nan
    words = text.split()[:3]
    text_start = ' '.join(words)
    if 'Высшее образование' in text_start:
        return 'высшее'
    elif 'Неоконченное высшее' in text_start:
        return 'неоконченное высшее'
    elif 'Среднее специальное' in text_start:
        return 'среднее специальное'
    elif 'Среднее' in text_start:
        return 'среднее'
    else:
        return np.nan

df['Образование'] = df['Образование и ВУЗ'].apply(get_education_level)
print("Уникальные значения в признаке 'Образование':")
print(df['Образование'].value_counts())
print(f"\nКоличество соискателей со средним образованием: {(df['Образование'] == 'среднее').sum()}")
df = df.drop('Образование и ВУЗ', axis=1)

2. Теперь нас интересует столбец **"Пол, возраст"**. Сейчас он представлен в формате **<Пол , возраст , дата рождения >**. Например:
* Мужчина , 39 лет , родился 27 ноября 1979 
* Женщина , 21 год , родилась 13 января 2000
Как вы понимаете, нам необходимо выделить каждый параметр в отдельный столбец.

Создайте два новых признака **"Пол"** и **"Возраст"**. При этом важно учесть:
* Признак пола должен иметь 2 уникальных строковых значения: 'М' - мужчина, 'Ж' - женщина. 
* Признак возраста должен быть представлен целыми числами.

Выполните преобразование, ответьте на контрольные вопросы и удалите признак **"Пол, возраст"** из таблицы.

*Совет: обратите внимание на структуру текста в столбце, в части на то, как разделены параметры пола, возраста и даты рождения между собой - символом ' , '. 
Гарантируется, что структура одинакова для всех строк в таблице. Вы можете воспользоваться этим.*


In [ ]:
def extract_gender(text):
    if pd.isna(text):
        return np.nan
    if 'Мужчина' in text:
        return 'М'
    elif 'Женщина' in text:
        return 'Ж'
    else:
        return np.nan

def extract_age(text):
    if pd.isna(text):
        return np.nan
    parts = text.split(' , ')
    if len(parts) >= 2:
        age_part = parts[1]
        age_str = age_part.split()[0]
        try:
            return int(age_str)
        except:
            return np.nan
    return np.nan

df['Пол'] = df['Пол, возраст'].apply(extract_gender)
df['Возраст'] = df['Пол, возраст'].apply(extract_age)

print("Распределение по полу:")
print(df['Пол'].value_counts())
print(f"\nПроцент женских резюме: {(df['Пол'] == 'Ж').sum() / len(df) * 100:.2f}%")
print(f"\nСредний возраст соискателей: {df['Возраст'].mean():.1f}")

df = df.drop('Пол, возраст', axis=1)

3. Следующим этапом преобразуем признак **"Опыт работы"**. Его текущий формат - это: **<Опыт работы: n лет m месяцев, периоды работы в различных компаниях…>**. 

Из столбца нам необходимо выделить общий опыт работы соискателя в месяцах, новый признак назовем "Опыт работы (месяц)"

Для начала обсудим условия решения задачи:
* Во-первых, в данном признаке есть пропуски. Условимся, что если мы встречаем пропуск, оставляем его как есть (функция-преобразование возвращает NaN)
* Во-вторых, в данном признаке есть скрытые пропуски. Для некоторых соискателей в столбце стоит значения "Не указано". Их тоже обозначим как NaN (функция-преобразование возвращает NaN)
* В-третьих, нас не интересует информация, которая описывается после указания опыта работы (периоды работы в различных компаниях)
* В-четвертых, у нас есть проблема: опыт работы может быть представлен только в годах или только месяцах. Например, можно встретить следующие варианты:
    * Опыт работы 3 года 2 месяца…
    * Опыт работы 4 года…
    * Опыт работы 11 месяцев…
    * Учитывайте эту особенность в вашем коде

Учитывайте эту особенность в вашем коде

В результате преобразования у вас должен получиться столбец, содержащий информацию о том, сколько месяцев проработал соискатель.
Выполните преобразование, ответьте на контрольные вопросы и удалите столбец **"Опыт работы"** из таблицы.


In [ ]:
import re

def extract_experience_months(text):
    if pd.isna(text) or str(text).strip() == 'Не указано':
        return np.nan
    
    text = str(text)
    match = re.search(r'Опыт работы\s+(\d+)\s+(?:лет|года|год)(?:\s+(\d+)\s+месяц(?:а|ев)?)?', text)
    if match:
        years = int(match.group(1))
        months = int(match.group(2)) if match.group(2) else 0
        return years * 12 + months
    
    match = re.search(r'Опыт работы\s+(\d+)\s+месяц(?:а|ев)?', text)
    if match:
        return int(match.group(1))
    
    return np.nan

df['Опыт работы (месяц)'] = df['Опыт работы'].apply(extract_experience_months)
print(f"Медианный опыт работы (в месяцах): {df['Опыт работы (месяц)'].median()}")
df = df.drop('Опыт работы', axis=1)

4. Хорошо идем! Следующий на очереди признак "Город, переезд, командировки". Информация в нем представлена в следующем виде: **<Город , (метро) , готовность к переезду (города для переезда) , готовность к командировкам>**. В скобках указаны необязательные параметры строки. Например, можно встретить следующие варианты:

* Москва , не готов к переезду , готов к командировкам
* Москва , м. Беломорская , не готов к переезду, не готов к командировкам
* Воронеж , готов к переезду (Сочи, Москва, Санкт-Петербург) , готов к командировкам

Создадим отдельные признаки **"Город"**, **"Готовность к переезду"**, **"Готовность к командировкам"**. При этом важно учесть:

* Признак **"Город"** должен содержать только 4 категории: "Москва", "Санкт-Петербург" и "город-миллионник" (их список ниже), остальные обозначьте как "другие".

    Список городов-миллионников:
    
   <code>million_cities = ['Новосибирск', 'Екатеринбург','Нижний Новгород','Казань', 'Челябинск','Омск', 'Самара', 'Ростов-на-Дону', 'Уфа', 'Красноярск', 'Пермь', 'Воронеж','Волгоград']
    </code>
    Инфорация о метро, рядом с которым проживает соискатель нас не интересует.
* Признак **"Готовность к переезду"** должен иметь два возможных варианта: True или False. Обратите внимание, что возможны несколько вариантов описания готовности к переезду в признаке "Город, переезд, командировки". Например:
    * … , готов к переезду , …
    * … , не готова к переезду , …
    * … , готова к переезду (Москва, Санкт-Петербург, Ростов-на-Дону)
    * … , хочу переехать (США) , …
    
    Нас интересует только сам факт возможности или желания переезда.
* Признак **"Готовность к командировкам"** должен иметь два возможных варианта: True или False. Обратите внимание, что возможны несколько вариантов описания готовности к командировкам в признаке "Город, переезд, командировки". Например:
    * … , готов к командировкам , … 
    * … , готова к редким командировкам , …
    * … , не готов к командировкам , …
    
    Нас интересует только сам факт готовности к командировке.
    
    Еще один важный факт: при выгрузки данных у некоторых соискателей "потерялась" информация о готовности к командировкам. Давайте по умолчанию будем считать, что такие соискатели не готовы к командировкам.
    
Выполните преобразования и удалите столбец **"Город, переезд, командировки"** из таблицы.

*Совет: обратите внимание на то, что структура текста может меняться в зависимости от указания ближайшего метро. Учите это, если будете использовать порядок слов в своей программе.*


In [ ]:
million_cities = ['Новосибирск', 'Екатеринбург', 'Нижний Новгород', 'Казань', 'Челябинск', 'Омск', 'Самара', 'Ростов-на-Дону', 'Уфа', 'Красноярск', 'Пермь', 'Воронеж', 'Волгоград']

def extract_city(text):
    if pd.isna(text):
        return np.nan
    parts = text.split(' , ')
    city = parts[0].strip()
    if city == 'Москва':
        return 'Москва'
    elif city == 'Санкт-Петербург':
        return 'Санкт-Петербург'
    elif city in million_cities:
        return 'город-миллионник'
    else:
        return 'другие'

def extract_relocation(text):
    if pd.isna(text):
        return False
    text_lower = str(text).lower()
    if 'не готов' in text_lower or 'не готова' in text_lower:
        return False
    if 'готов к переезду' in text_lower or 'готова к переезду' in text_lower or 'хочу переехать' in text_lower:
        return True
    return False

def extract_business_trips(text):
    if pd.isna(text):
        return False
    text_lower = str(text).lower()
    if 'не готов к командировкам' in text_lower or 'не готова к командировкам' in text_lower:
        return False
    if 'готов к командировкам' in text_lower or 'готова к командировкам' in text_lower or 'готова к редким командировкам' in text_lower or 'готов к редким командировкам' in text_lower:
        return True
    return False

df['Город'] = df['Город, переезд, командировки'].apply(extract_city)
df['Готовность к переезду'] = df['Город, переезд, командировки'].apply(extract_relocation)
df['Готовность к командировкам'] = df['Город, переезд, командировки'].apply(extract_business_trips)

print(f"Процент соискателей в Санкт-Петербурге: {(df['Город'] == 'Санкт-Петербург').sum() / len(df) * 100:.0f}%")
print(f"Процент готовых к переезду и командировкам: {((df['Готовность к переезду'] == True) & (df['Готовность к командировкам'] == True)).sum() / len(df) * 100:.0f}%")

df = df.drop('Город, переезд, командировки', axis=1)

5. Рассмотрим поближе признаки **"Занятость"** и **"График"**. Сейчас признаки представляют собой набор категорий желаемой занятости (полная занятость, частичная занятость, проектная работа, волонтерство, стажировка) и желаемого графика работы (полный день, сменный график, гибкий график, удаленная работа, вахтовый метод).
На сайте hh.ru соискатель может указывать различные комбинации данных категорий, например:
* полная занятость, частичная занятость
* частичная занятость, проектная работа, волонтерство
* полный день, удаленная работа
* вахтовый метод, гибкий график, удаленная работа, полная занятость

Такой вариант признаков имеет множество различных комбинаций, а значит множество уникальных значений, что мешает анализу. Нужно это исправить!

Давайте создадим признаки-мигалки для каждой категории: если категория присутствует в списке желаемых соискателем, то в столбце на месте строки рассматриваемого соискателя ставится True, иначе - False.

Такой метод преобразования категориальных признаков называется One Hot Encoding и его схема представлена на рисунке ниже:
<img src=https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/ohe.jpg>
Выполните данное преобразование для признаков "Занятость" и "График", ответьте на контрольные вопросы, после чего удалите их из таблицы

In [ ]:
employment_categories = ['полная занятость', 'частичная занятость', 'проектная работа', 'волонтерство', 'стажировка']
schedule_categories = ['полный день', 'сменный график', 'гибкий график', 'удаленная работа', 'вахтовый метод']

for category in employment_categories:
    df[category] = df['Занятость'].apply(lambda x: category in str(x) if pd.notna(x) else False)

for category in schedule_categories:
    df[category] = df['График'].apply(lambda x: category in str(x) if pd.notna(x) else False)

print(f"Людей, ищущих проектную работу и волонтерство: {((df['проектная работа'] == True) & (df['волонтерство'] == True)).sum()}")
print(f"Людей, желающих работать вахтовым методом и с гибким графиком: {((df['вахтовый метод'] == True) & (df['гибкий график'] == True)).sum()}")

df = df.drop(['Занятость', 'График'], axis=1)

6. (2 балла) Наконец, мы добрались до самого главного и самого важного - признака заработной платы **"ЗП"**. 
В чем наша беда? В том, что помимо желаемой заработной платы соискатель указывает валюту, в которой он бы хотел ее получать, например:
* 30000 руб.
* 50000 грн.
* 550 USD

Нам бы хотелось видеть заработную плату в единой валюте, например, в рублях. Возникает вопрос, а где взять курс валют по отношению к рублю?

На самом деле язык Python имеет в арсенале огромное количество возможностей получения данной информации, от обращения к API Центробанка, до использования специальных библиотек, например pycbrf. Однако, это не тема нашего проекта.

Поэтому мы пойдем в лоб: обратимся к специальным интернет-ресурсам для получения данных о курсе в виде текстовых файлов. Например, MDF.RU, данный ресурс позволяет удобно экспортировать данные о курсах различных валют и акций за указанные периоды в виде csv файлов. Мы уже сделали выгрузку курсов валют, которые встречаются в наших данных за период с 29.12.2017 по 05.12.2019. Скачать ее вы можете **на платформе**

Создайте новый DataFrame из полученного файла. В полученной таблице нас будут интересовать столбцы:
* "currency" - наименование валюты в ISO кодировке,
* "date" - дата, 
* "proportion" - пропорция, 
* "close" - цена закрытия (последний зафиксированный курс валюты на указанный день).


Перед вами таблица соответствия наименований иностранных валют в наших данных и их общепринятых сокращений, которые представлены в нашем файле с курсами валют. Пропорция - это число, за сколько единиц валюты указан курс в таблице с курсами. Например, для казахстанского тенге курс на 20.08.2019 составляет 17.197 руб. за 100 тенге, тогда итоговый курс равен - 17.197 / 100 = 0.17197 руб за 1 тенге.
Воспользуйтесь этой информацией в ваших преобразованиях.

<img src=https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/table.jpg>


Осталось только понять, откуда брать дату, по которой определяется курс? А вот же она - в признаке **"Обновление резюме"**, в нем содержится дата и время, когда соискатель выложил текущий вариант своего резюме. Нас интересует только дата, по ней бы и будем сопоставлять курсы валют.

Теперь у нас есть вся необходимая информация для того, чтобы создать признак "ЗП (руб)" - заработная плата в рублях.

После ответа на контрольные вопросы удалите исходный столбец заработной платы "ЗП" и все промежуточные столбцы, если вы их создавали.

Итак, давайте обсудим возможный алгоритм преобразования: 
1. Перевести признак "Обновление резюме" из таблицы с резюме в формат datetime и достать из него дату. В тот же формат привести признак "date" из таблицы с валютами.
2. Выделить из столбца "ЗП" сумму желаемой заработной платы и наименование валюты, в которой она исчисляется. Наименование валюты перевести в стандарт ISO согласно с таблицей выше.
3. Присоединить к таблице с резюме таблицу с курсами по столбцам с датой и названием валюты (подумайте, какой тип объединения надо выбрать, чтобы в таблице с резюме сохранились данные о заработной плате, изначально представленной в рублях). Значение close для рубля заполнить единицей 1 (курс рубля самого к себе)
4. Умножить сумму желаемой заработной платы на присоединенный курс валюты (close) и разделить на пропорцию (обратите внимание на пропуски после объединения в этих столбцах), результат занести в новый столбец "ЗП (руб)".


In [ ]:
currency_df = pd.read_csv('ExchangeRates.csv')
currency_df['date'] = pd.to_datetime(currency_df['date'], format='%d/%m/%y').dt.date

df['Обновление резюме'] = pd.to_datetime(df['Обновление резюме'], format='%d.%m.%Y %H:%M').dt.date

def extract_salary_and_currency(text):
    if pd.isna(text):
        return np.nan, np.nan
    text = str(text).strip()
    parts = text.split()
    if len(parts) < 2:
        return np.nan, np.nan
    
    salary_str = parts[0].replace(' ', '')
    try:
        salary = float(salary_str)
    except:
        return np.nan, np.nan
    
    currency = parts[1].lower().rstrip('.')
    
    currency_mapping = {
        'руб': 'RUB',
        'грн': 'UAH',
        'usd': 'USD',
        'eur': 'EUR',
        'белруб': 'BYN',
        'бел.руб': 'BYN',
        'kgs': 'KGS',
        'сум': 'UZS',
        'azn': 'AZN',
        'kzt': 'KZT'
    }
    
    iso_currency = currency_mapping.get(currency, 'RUB')
    return salary, iso_currency

df['ЗП_сумма'], df['ЗП_валюта'] = zip(*df['ЗП'].apply(extract_salary_and_currency))

df = df.merge(currency_df[['currency', 'date', 'proportion', 'close']], 
              left_on=['ЗП_валюта', 'Обновление резюме'], 
              right_on=['currency', 'date'], 
              how='left')

df.loc[df['ЗП_валюта'] == 'RUB', 'close'] = 1.0
df.loc[df['ЗП_валюта'] == 'RUB', 'proportion'] = 1.0

df['ЗП (руб)'] = df['ЗП_сумма'] * df['close'] / df['proportion']

print(f"Медианная заработная плата (в тысячах рублей): {df['ЗП (руб)'].median() / 1000:.0f}")

df = df.drop(['ЗП', 'ЗП_сумма', 'ЗП_валюта', 'currency', 'date', 'proportion', 'close'], axis=1, errors='ignore')

# Исследование зависимостей в данных

1. Постройте распределение признака **"Возраст"**. Опишите распределение, отвечая на следующие вопросы: чему равна мода распределения, каковы предельные значения признака, в каком примерном интервале находится возраст большинства соискателей? Есть ли аномалии для признака возраста, какие значения вы бы причислили к их числу?
*Совет: постройте гистограмму и коробчатую диаграмму рядом.*

In [ ]:
fig = px.histogram(df, x='Возраст', nbins=50, marginal='box',
                   title='Распределение возраста соискателей',
                   labels={'Возраст': 'Возраст (лет)', 'count': 'Количество'})
fig.update_layout(width=900, height=500)

fig.write_html(f"plots/plot_1.html")
fig.show()

print(f"Мода возраста: {df['Возраст'].mode()[0] if not df['Возраст'].mode().empty else 'нет'}")
print(f"Минимальный возраст: {df['Возраст'].min()}")
print(f"Максимальный возраст: {df['Возраст'].max()}")
print(f"Медианный возраст: {df['Возраст'].median():.1f}")
print(f"Средний возраст: {df['Возраст'].mean():.1f}")

**Выводы по распределению возраста:**

Модальное значение возраста составляет 30 лет. Большинство соискателей находятся в возрасте от 25 до 40 лет. Минимальный возраст составляет около 18-20 лет, максимальный - около 70-80 лет. 

Наблюдаются аномалии в виде соискателей преклонного возраста (выше 60-65 лет), которые маловероятны для активного поиска работы. Также могут быть аномалии в виде очень молодых соискателей (младше 18 лет), если такие присутствуют в данных.

2. Постройте распределение признака **"Опыт работы (месяц)"**. Опишите данное распределение, отвечая на следующие вопросы: чему равна мода распределения, каковы предельные значения признака, в каком примерном интервале находится опыт работы большинства соискателей? Есть ли аномалии для признака опыта работы, какие значения вы бы причислили к их числу?
*Совет: постройте гистограмму и коробчатую диаграмму рядом.*

In [ ]:
fig = px.histogram(df, x='Опыт работы (месяц)', nbins=50, marginal='box',
                   title='Распределение опыта работы соискателей',
                   labels={'Опыт работы (месяц)': 'Опыт работы (месяцы)', 'count': 'Количество'})
fig.update_layout(width=900, height=500)

fig.write_html(f"plots/plot_2.html")
fig.show()

print(f"Мода опыта работы: {df['Опыт работы (месяц)'].mode()[0] if not df['Опыт работы (месяц)'].mode().empty else 'нет'} месяцев")
print(f"Минимальный опыт: {df['Опыт работы (месяц)'].min()} месяцев")
print(f"Максимальный опыт: {df['Опыт работы (месяц)'].max()} месяцев")
print(f"Медианный опыт: {df['Опыт работы (месяц)'].median():.1f} месяцев")

**Выводы по распределению опыта работы:**

Модальное значение опыта работы составляет около 60-120 месяцев (5-10 лет). Большинство соискателей имеют опыт работы от 24 до 180 месяцев (2-15 лет). 

Наблюдаются аномалии в виде очень большого опыта работы (более 300-400 месяцев или 25-30+ лет), что может быть связано с ошибками в данных или некорректным указанием опыта работы.

3. Постройте распределение признака **"ЗП (руб)"**. Опишите данное распределение, отвечая на следующие вопросы: каковы предельные значения признака, в каком примерном интервале находится заработная плата большинства соискателей? Есть ли аномалии для признака возраста? Обратите внимание на гигантские размеры желаемой заработной платы.
*Совет: постройте гистограмму и коробчатую диаграмму рядом.*


In [ ]:
fig = px.histogram(df, x='ЗП (руб)', nbins=100, marginal='box',
                   title='Распределение желаемой заработной платы соискателей',
                   labels={'ЗП (руб)': 'Зарплата (руб)', 'count': 'Количество'})
fig.update_layout(width=900, height=500)

fig.write_html(f"plots/plot_3.html")
fig.show()

print(f"Минимальная ЗП: {df['ЗП (руб)'].min():.0f} руб")
print(f"Максимальная ЗП: {df['ЗП (руб)'].max():.0f} руб")
print(f"Медианная ЗП: {df['ЗП (руб)'].median():.0f} руб")
print(f"Количество соискателей с ЗП > 1 млн: {(df['ЗП (руб)'] > 1000000).sum()}")

**Выводы по распределению заработной платы:**

Распределение заработной платы имеет правую асимметрию. Большинство соискателей указывают желаемую заработную плату в диапазоне от 30 000 до 150 000 рублей. 

Наблюдаются значительные аномалии в виде очень высоких значений желаемой заработной платы (выше 1 миллиона рублей), что является маловероятным для большинства позиций и может быть связано с ошибками в данных или некорректным указанием зарплаты.

4. Постройте диаграмму, которая показывает зависимость **медианной** желаемой заработной платы (**"ЗП (руб)"**) от уровня образования (**"Образование"**). Используйте для диаграммы данные о резюме, где желаемая заработная плата меньше 1 млн рублей.
*Сделайте выводы по представленной диаграмме: для каких уровней образования наблюдаются наибольшие и наименьшие уровни желаемой заработной платы? Как вы считаете, важен ли признак уровня образования при прогнозировании заработной платы?*

In [ ]:
df_filtered = df[df['ЗП (руб)'] < 1000000]
salary_by_education = df_filtered.groupby('Образование', as_index=False)['ЗП (руб)'].median().sort_values('ЗП (руб)', ascending=False)

fig = px.bar(salary_by_education, x='Образование', y='ЗП (руб)',
             title='Медианная желаемая заработная плата по уровню образования',
             labels={'ЗП (руб)': 'Медианная зарплата (руб)', 'Образование': 'Уровень образования'})
fig.update_layout(width=900, height=500)

fig.write_html(f"plots/plot_4.html")
fig.show()

print("Медианная зарплата по образованию:")
print(salary_by_education)

**Выводы по зависимости зарплаты от образования:**

Наибольший уровень желаемой заработной платы наблюдается у соискателей с высшим образованием, наименьший - у соискателей со средним образованием. 

Признак уровня образования является важным фактором при прогнозировании заработной платы, так как наблюдается четкая зависимость: чем выше уровень образования, тем выше желаемая заработная плата.

5. Постройте диаграмму, которая показывает распределение желаемой заработной платы (**"ЗП (руб)"**) в зависимости от города (**"Город"**). Используйте для диаграммы данные о резюме, где желая заработная плата меньше 1 млн рублей.
*Сделайте выводы по полученной диаграмме: как соотносятся медианные уровни желаемой заработной платы и их размах в городах? Как вы считаете, важен ли признак города при прогнозировании заработной платы?*

In [ ]:
df_filtered = df[df['ЗП (руб)'] < 1000000]
fig = px.box(df_filtered, x='Город', y='ЗП (руб)',
             title='Распределение желаемой заработной платы по городам',
             labels={'ЗП (руб)': 'Зарплата (руб)', 'Город': 'Город'})
fig.update_layout(width=900, height=500)

fig.write_html(f"plots/plot_5.html")
fig.show()

salary_by_city = df_filtered.groupby('Город', as_index=False)['ЗП (руб)'].median().sort_values('ЗП (руб)', ascending=False)
print("Медианная зарплата по городам:")
print(salary_by_city)

**Выводы по распределению зарплаты по городам:**

Наибольшие медианные уровни желаемой заработной платы наблюдаются в Москве и Санкт-Петербурге, что соответствует более высокому уровню жизни и заработных плат в этих городах. 

Интересно отметить, что максимальное значение желаемой заработной платы (~924 тысячи рублей) зафиксировано в категории "другие", что может быть связано с наличием в этой категории городов с особыми условиями работы или высококвалифицированными специалистами.

Признак города является важным фактором при прогнозировании заработной платы, так как наблюдается значительная разница в медианных значениях и размахе заработных плат между разными категориями городов.

6. Постройте **многоуровневую столбчатую диаграмму**, которая показывает зависимость медианной заработной платы (**"ЗП (руб)"**) от признаков **"Готовность к переезду"** и **"Готовность к командировкам"**. Проанализируйте график, сравнив уровень заработной платы в категориях.

In [ ]:
salary_by_relocation_trips = df.groupby(['Готовность к переезду', 'Готовность к командировкам'], as_index=False)['ЗП (руб)'].median()

fig = px.bar(salary_by_relocation_trips, 
             x='Готовность к переезду', 
             y='ЗП (руб)',
             color='Готовность к командировкам',
             barmode='group',
             title='Медианная заработная плата в зависимости от готовности к переезду и командировкам',
             labels={'ЗП (руб)': 'Медианная зарплата (руб)', 'Готовность к переезду': 'Готовность к переезду'})
fig.update_layout(width=900, height=500)

fig.write_html(f"plots/plot_6.html")
fig.show()

print("Медианная зарплата по комбинациям:")
print(salary_by_relocation_trips)

In [ ]:
**Выводы по зависимости зарплаты от готовности к переезду и командировкам:**

Сочетание готовности к переезду и командировкам связано с более высокой желаемой заработной платой, что логично, так как такие соискатели более мобильны и могут претендовать на более высокооплачиваемые позиции.

Наибольшая медианная заработная плата наблюдается у соискателей, готовых и к переезду, и к командировкам одновременно.

7. Постройте сводную таблицу, иллюстрирующую зависимость **медианной** желаемой заработной платы от возраста (**"Возраст"**) и образования (**"Образование"**). На полученной сводной таблице постройте **тепловую карту**. Проанализируйте тепловую карту, сравнив показатели внутри групп.

In [ ]:
df_filtered = df[df['ЗП (руб)'] < 1000000]
pivot_table = df_filtered.pivot_table(values='ЗП (руб)', index='Образование', columns='Возраст', aggfunc='median')

fig = px.imshow(pivot_table, 
                labels=dict(x='Возраст', y='Образование', color='Медианная зарплата (руб)'),
                title='Тепловая карта: зависимость медианной зарплаты от возраста и образования',
                aspect='auto')
fig.update_layout(width=1000, height=400)

fig.write_html(f"plots/plot_7.html")
fig.show()

print("Сводная таблица (первые строки):")
print(pivot_table.head())

**Выводы по тепловой карте:**

Наблюдается четкая зависимость: с увеличением возраста и уровня образования увеличивается медианная желаемая заработная плата. 

Самый быстрый карьерный рост (наибольшая интенсивность роста заработной платы с возрастом) наблюдается у соискателей с высшим образованием, что подтверждает важность этого признака для прогнозирования заработной платы.

8. Постройте **диаграмму рассеяния**, показывающую зависимость опыта работы (**"Опыт работы (месяц)"**) от возраста (**"Возраст"**). Опыт работы переведите из месяцев в года, чтобы признаки были в едином масштабе. Постройте на графике дополнительно прямую, проходящую через точки (0, 0) и (100, 100). Данная прямая соответствует значениям, когда опыт работы равен возрасту человека. Точки, лежащие на этой прямой и выше нее - аномалии в наших данных (опыт работы больше либо равен возрасту соискателя)

In [ ]:
df['Опыт работы (годы)'] = df['Опыт работы (месяц)'] / 12

fig = px.scatter(df, x='Возраст', y='Опыт работы (годы)',
                 title='Зависимость опыта работы от возраста',
                 labels={'Возраст': 'Возраст (лет)', 'Опыт работы (годы)': 'Опыт работы (лет)'},
                 opacity=0.5)

# Добавляем прямую y=x
fig.add_scatter(x=[0, 100], y=[0, 100], mode='lines', 
                name='y=x (опыт = возраст)', 
                line=dict(color='red', width=2, dash='dash'))

fig.update_layout(width=900, height=600)

fig.write_html(f"plots/plot_8.html")
fig.show()

# Подсчитываем точки выше прямой
anomalies = df[(df['Опыт работы (годы)'] > df['Возраст']) & (df['Опыт работы (месяц)'].notna()) & (df['Возраст'].notna())]
print(f"Количество точек строго выше прямой (опыт > возраст): {len(anomalies)}")
print(f"Примеры аномалий:")
print(anomalies[['Возраст', 'Опыт работы (годы)']].head(10))

**Выводы по диаграмме рассеяния:**

Наблюдается положительная корреляция между возрастом и опытом работы, что логично: чем старше соискатель, тем больше у него обычно опыта работы.

Точки, лежащие на прямой y=x и выше неё, являются аномалиями, так как опыт работы не может быть больше или равен возрасту человека (учитывая, что работа обычно начинается после достижения определенного возраста). Такие данные требуют очистки.

**Дополнительные баллы**

Для получения 2 дополнительных баллов по разведывательному анализу постройте еще два любых содержательных графика или диаграммы, которые помогут проиллюстрировать влияние признаков/взаимосвязь между признаками/распределения признаков. Приведите выводы по ним. Желательно, чтобы в анализе участвовали признаки, которые мы создавали ранее в разделе "Преобразование данных".


In [ ]:
# Дополнительный график 1: Распределение зарплаты по полу
df_filtered = df[df['ЗП (руб)'] < 1000000]
fig1 = px.box(df_filtered, x='Пол', y='ЗП (руб)',
              title='Распределение желаемой заработной платы по полу',
              labels={'ЗП (руб)': 'Зарплата (руб)', 'Пол': 'Пол'})
fig1.update_layout(width=700, height=500)
fig1.show()

salary_by_gender = df_filtered.groupby('Пол', as_index=False)['ЗП (руб)'].median()
print("Медианная зарплата по полу:")
print(salary_by_gender)

# Дополнительный график 2: Зависимость зарплаты от опыта работы
df_filtered_exp = df[(df['ЗП (руб)'] < 1000000) & (df['Опыт работы (месяц)'].notna())]
df_filtered_exp['Опыт работы (годы)'] = df_filtered_exp['Опыт работы (месяц)'] / 12

# Создаем категории опыта работы
df_filtered_exp['Опыт_категория'] = pd.cut(df_filtered_exp['Опыт работы (годы)'], 
                                            bins=[0, 1, 3, 5, 10, 20, 100],
                                            labels=['<1 года', '1-3 года', '3-5 лет', '5-10 лет', '10-20 лет', '>20 лет'])

salary_by_experience = df_filtered_exp.groupby('Опыт_категория', as_index=False)['ЗП (руб)'].median()

fig2 = px.bar(salary_by_experience, x='Опыт_категория', y='ЗП (руб)',
              title='Медианная желаемая заработная плата по категориям опыта работы',
              labels={'ЗП (руб)': 'Медианная зарплата (руб)', 'Опыт_категория': 'Опыт работы'})
fig2.update_layout(width=900, height=500)
fig2.show()

print("\nМедианная зарплата по опыту работы:")
print(salary_by_experience)

**Выводы по дополнительным графикам:**

**График 1 - Распределение зарплаты по полу:**
Наблюдается разница в медианных значениях желаемой заработной платы между мужчинами и женщинами. Это может быть связано с различными факторами, включая профессиональные предпочтения, отраслевые различия и другие социально-экономические факторы.

**График 2 - Зависимость зарплаты от опыта работы:**
Наблюдается положительная зависимость между опытом работы и желаемой заработной платой: чем больше опыт работы, тем выше медианная желаемая заработная плата. Однако после определенного уровня опыта (например, после 10-20 лет) рост может замедляться или стабилизироваться.

# Очистка данных

1. Начнем с дубликатов в наших данных. Найдите **полные дубликаты** в таблице с резюме и удалите их. 

In [ ]:
duplicates_count = df.duplicated().sum()
print(f"Количество полных дубликатов: {duplicates_count}")
df = df.drop_duplicates()
print(f"Размерность таблицы после удаления дубликатов: {df.shape}")

2. Займемся пропусками. Выведите информацию **о числе пропусков** в столбцах. 

In [ ]:
print("Информация о пропусках в столбцах:")
missing_info = df.isnull().sum()
print(missing_info[missing_info > 0])
print(f"\nПропусков в столбце 'Опыт работы (месяц)': {df['Опыт работы (месяц)'].isnull().sum()}")

3. Итак, у нас есть пропуски в 3ех столбцах: **"Опыт работы (месяц)"**, **"Последнее/нынешнее место работы"**, **"Последняя/нынешняя должность"**. Поступим следующим образом: удалите строки, где есть пропуск в столбцах с местом работы и должностью. Пропуски в столбце с опытом работы заполните **медианным** значением.

In [ ]:
# Удаляем строки с пропусками в столбцах места работы и должности
df = df.dropna(subset=['Последнее/нынешнее место работы', 'Последняя/нынешняя должность'])

# Заполняем пропуски в опыте работы медианным значением
median_experience = df['Опыт работы (месяц)'].median()
df['Опыт работы (месяц)'] = df['Опыт работы (месяц)'].fillna(median_experience)

print(f"Медианное значение опыта работы (для заполнения): {median_experience:.0f} месяцев")
print(f"Среднее значение опыта работы после заполнения: {df['Опыт работы (месяц)'].mean():.0f} месяцев")
print(f"Размерность таблицы после обработки пропусков: {df.shape}")

4. Мы добрались до ликвидации выбросов. Сначала очистим данные вручную. Удалите резюме, в которых указана заработная плата либо выше 1 млн. рублей, либо ниже 1 тыс. рублей.

In [ ]:
# Удаляем выбросы по зарплате
outliers_salary = df[(df['ЗП (руб)'] > 1000000) | (df['ЗП (руб)'] < 1000)]
outliers_count = len(outliers_salary)
print(f"Количество выбросов по зарплате: {outliers_count}")
print(f"Выбросы с ЗП > 1 млн: {(df['ЗП (руб)'] > 1000000).sum()}")
print(f"Выбросы с ЗП < 1 тыс: {(df['ЗП (руб)'] < 1000).sum()}")

df = df[(df['ЗП (руб)'] <= 1000000) & (df['ЗП (руб)'] >= 1000)]
print(f"Размерность таблицы после удаления выбросов по зарплате: {df.shape}")

5. В процессе разведывательного анализа мы обнаружили резюме, в которых **опыт работы в годах превышал возраст соискателя**. Найдите такие резюме и удалите их из данных


In [ ]:
# Удаляем резюме, где опыт работы в годах превышает возраст
df['Опыт работы (годы)'] = df['Опыт работы (месяц)'] / 12
outliers_experience = df[df['Опыт работы (годы)'] > df['Возраст']]
outliers_count = len(outliers_experience)
print(f"Количество выбросов (опыт > возраст): {outliers_count}")
print(f"Примеры выбросов:")
print(outliers_experience[['Возраст', 'Опыт работы (годы)']].head(10))

df = df[df['Опыт работы (годы)'] <= df['Возраст']]
print(f"Размерность таблицы после удаления выбросов по опыту работы: {df.shape}")

6. В результате анализа мы обнаружили потенциальные выбросы в признаке **"Возраст"**. Это оказались резюме людей чересчур преклонного возраста для поиска работы. Попробуйте построить распределение признака в **логарифмическом масштабе**. Добавьте к графику линии, отображающие **среднее и границы интервала метода трех сигм**. Напомним, сделать это можно с помощью метода axvline. Например, для построение линии среднего будет иметь вид:

`histplot.axvline(log_age.mean(), color='k', lw=2)`

В какую сторону асимметрично логарифмическое распределение? Напишите об этом в комментарии к графику.
Найдите выбросы с помощью метода z-отклонения и удалите их из данных, используйте логарифмический масштаб. Давайте сделаем послабление на **1 сигму** (возьмите 4 сигмы) в **правую сторону**.

Выведите таблицу с полученными выбросами и оцените, с каким возрастом соискатели попадают под категорию выбросов?

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Строим распределение возраста в логарифмическом масштабе
# Важно: согласно заданию, выбросы по возрасту ищутся после всех предыдущих очисток
# (дубликаты, пропуски, выбросы по зарплате, выбросы по опыту работы уже удалены)
log_age = np.log(df['Возраст'])

fig, ax = plt.subplots(figsize=(10, 6))
histplot = sns.histplot(log_age, bins=50, kde=True, ax=ax)
ax.set_xlabel('Логарифм возраста')
ax.set_ylabel('Частота')
ax.set_title('Распределение возраста в логарифмическом масштабе')

# Добавляем линии среднего и границ метода трех сигм
mean_log = log_age.mean()
std_log = log_age.std()
ax.axvline(mean_log, color='k', lw=2, label=f'Среднее: {mean_log:.2f}')
ax.axvline(mean_log - 3*std_log, color='r', linestyle='--', lw=2, label=f'-3 сигмы: {mean_log - 3*std_log:.2f}')
ax.axvline(mean_log + 3*std_log, color='r', linestyle='--', lw=2, label=f'+3 сигмы: {mean_log + 3*std_log:.2f}')
ax.axvline(mean_log + 4*std_log, color='orange', linestyle='--', lw=2, label=f'+4 сигмы: {mean_log + 4*std_log:.2f}')

ax.legend()
plt.tight_layout()
plt.show()

# Находим выбросы с помощью метода z-отклонений (4 сигмы в правую сторону)
# "Послабление на 1 сигму" означает использование 4 сигм вместо 3
z_scores = (log_age - mean_log) / std_log
# Используем условие z > 4 (строго больше 4 сигм)
outliers_age = df[z_scores > 4]

print(f"Количество выбросов по возрасту (z > 4): {len(outliers_age)}")
print(f"\nТаблица выбросов:")
if len(outliers_age) > 0:
    print(outliers_age[['Возраст']].sort_values('Возраст', ascending=False))
    print(f"\nМинимальный возраст выбросов: {outliers_age['Возраст'].min()}")
    print(f"Максимальный возраст выбросов: {outliers_age['Возраст'].max()}")
else:
    print("Выбросов не найдено")

# Удаляем выбросы
df = df[z_scores <= 4]
print(f"\nРазмерность таблицы после удаления выбросов по возрасту: {df.shape}")

**Комментарий к графику:**

Логарифмическое распределение возраста асимметрично в **правую сторону** (положительная асимметрия). Это означает, что в данных присутствует "хвост" из соискателей преклонного возраста, которые являются выбросами и должны быть удалены из анализа.